In [ ]:
import random
random.seed(109)
class Mancala:
    def __init__(self, pits_per_player=6, stones_per_pit = 4):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1]
        self.p2_mancala_index = len(self.board)-1
        
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
        
    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """
        if self.current_player == 1:
            pit_index = pit - 1
            if pit_index < self.p1_pits_index[0] or pit_index > self.p1_pits_index[1]:
                return False
        else:
            pit_index = self.p2_pits_index[1] - (pit - 1)
            if pit_index < self.p2_pits_index[0] or pit_index > self.p2_pits_index[1]:
                return False
        
        return self.board[pit_index] > 0
        
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        import random
        valid_pits = []
        for pit in range(1, self.pits_per_player + 1):
            if self.valid_move(pit):
                valid_pits.append(pit)
        return random.choice(valid_pits) if valid_pits else None
    
    def play(self, pit):
        """
        Simulated a play in Mancala
        """
        if not self.valid_move(pit):
            print("INVALID MOVE")
            return self.board
        
        if self.winning_eval():
            print("GAME OVER")
            return self.board
        
        self.moves.append((self.current_player, pit))
        
        if self.current_player == 1:
            pit_index = pit - 1
            my_pits_range = self.p1_pits_index
            my_mancala = self.p1_mancala_index
            opponent_mancala = self.p2_mancala_index
            opponent_pits_range = self.p2_pits_index
        else:
            pit_index = self.p2_pits_index[1] - (pit - 1)
            my_pits_range = self.p2_pits_index
            my_mancala = self.p2_mancala_index
            opponent_mancala = self.p1_mancala_index
            opponent_pits_range = self.p1_pits_index
        
        stones = self.board[pit_index]
        self.board[pit_index] = 0
        
        current_index = pit_index
        
        while stones > 0:
            current_index = (current_index + 1) % len(self.board)
            if current_index == opponent_mancala:
                continue
            self.board[current_index] += 1
            stones -= 1
        
        if current_index != my_mancala:
            if my_pits_range[0] <= current_index <= my_pits_range[1] and self.board[current_index] == 1:
                opposite_index = len(self.board) - 2 - current_index
                if self.board[opposite_index] > 0:
                    captured = self.board[current_index] + self.board[opposite_index]
                    self.board[current_index] = 0
                    self.board[opposite_index] = 0
                    self.board[my_mancala] += captured
            
            self.current_player = 2 if self.current_player == 1 else 1
        
        return self.board
    
    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        p1_pits_sum = sum(self.board[self.p1_pits_index[0]:self.p1_pits_index[1]+1])
        p2_pits_sum = sum(self.board[self.p2_pits_index[0]:self.p2_pits_index[1]+1])
        
        if p1_pits_sum == 0 or p2_pits_sum == 0:
            self.board[self.p1_mancala_index] += p1_pits_sum
            self.board[self.p2_mancala_index] += p2_pits_sum
            for i in range(self.p1_pits_index[0], self.p1_pits_index[1]+1):
                self.board[i] = 0
            for i in range(self.p2_pits_index[0], self.p2_pits_index[1]+1):
                self.board[i] = 0
            return True
        
        return False
    
    # Mancala part 2
import random

random.seed(42)

# init
game = Mancala(pits_per_player=6, stones_per_pit=4)

# 10 moves total 
moves_played = 0
max_moves = 10

while moves_played < max_moves and not game.winning_eval():
    # student
    if game.current_player == 1:
        student_moves = [1, 2, 3, 4, 5]  # Example moves for P1
        pit = student_moves[moves_played // 2] if moves_played // 2 < len(student_moves) else 1
    # random
    else:
        pit = game.random_move_generator()
        if pit is None:
            break
    
    game.play(pit)
    moves_played += 1
    
# results
print("Game completed after {} moves".format(moves_played))

if game.board[game.p1_mancala_index] > game.board[game.p2_mancala_index]:
    print("Student WINS")
elif game.board[game.p1_mancala_index] < game.board[game.p2_mancala_index]:
    print("Random WINS")
else:
    print("TIE")

: 